In [16]:
# Signal that the script started working
with open('train-status.txt', 'w') as file: file.write('0')

In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dropout, Dense
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import gc
import sys

In [18]:
data_train = np.load('dataset-split/batch-920.npy', allow_pickle=True) # Replace the string with sys.argv[0]
data = pd.DataFrame(data_train, columns=['features','labels'])

In [19]:
for i in data.index:
    data['features'][i] = data['features'][i].reshape(226, 226, 3)

In [20]:
from keras.utils import to_categorical

# Get the input data
X = np.stack(data['features'].to_numpy())

# Normalize
X = X / 255.0

# Get the labels
y = data['labels'].to_numpy()
y = y.astype(int)

# One-hot encode
y = np.vectorize(lambda x: x//5 if x>1 else x)(y)

# Subtract the minimum label value to ensure that label values start from 0
y = y - np.min(y)

# One-hot encode
y = to_categorical(y, num_classes=3)

In [21]:
# Split the data into training and testing sets (with stratification)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [22]:
del X
del y
gc.collect()

1465

In [23]:
# Load the saved model from the H5 file
model_cnn = load_model('cnn.h5')

# Compile the model
model_cnn.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

In [24]:
del data
del data_train
gc.collect()

598

In [25]:
# Final preparation
X_train, y_train, X_test, y_test = tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train), tf.convert_to_tensor(X_test), tf.convert_to_tensor(y_test)

# Train the model
model_cnn.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
1/1 [==============================] - 16s 16s/step - loss: 0.2191 - accuracy: 0.9167 - val_loss: 2.9883 - val_accuracy: 0.2500
Epoch 2/10
1/1 [==============================] - 15s 15s/step - loss: 0.8059 - accuracy: 0.6667 - val_loss: 5.3393 - val_accuracy: 0.2500
Epoch 3/10
1/1 [==============================] - 15s 15s/step - loss: 0.0504 - accuracy: 1.0000 - val_loss: 8.9847 - val_accuracy: 0.2500
Epoch 4/10
1/1 [==============================] - 15s 15s/step - loss: 0.1444 - accuracy: 0.9167 - val_loss: 12.4272 - val_accuracy: 0.2500
Epoch 5/10
1/1 [==============================] - 16s 16s/step - loss: 0.2689 - accuracy: 0.8333 - val_loss: 12.5868 - val_accuracy: 0.2500
Epoch 6/10
1/1 [==============================] - 15s 15s/step - loss: 0.0168 - accuracy: 1.0000 - val_loss: 12.7517 - val_accuracy: 0.2500
Epoch 7/10
1/1 [==============================] - 15s 15s/step - loss: 0.0077 - accuracy: 1.0000 - val_loss: 13.0528 - val_accuracy: 0.2500
Epoch 8/10
1/1 [=======

In [26]:
# Save the model
import os
model_cnn.save(os.path.join(os.getcwd(), 'cnn.h5'))

In [27]:
# Signal that the script finished
with open('train-status.txt', 'w') as file: file.write('1')